In [20]:
# Dependency Graph (DG) based prefetching technique
# Paper: Using Predictive Prefetching to Improve World Wide Web Latency, 1996
# Assumption: no cache staleness 
# Need to computer URL = host + uri (might not be the actual URL b/c some uris already have host and some don't, 
# but it doesn't affect the analysis results b/c the format is the same, e.g., the uri that starts with host name
# always starts with host name)

import networkx as nx
import matplotlib.pyplot as plt
import csv
import time as sys_time

window_size = 3
weight_threshold = 0.8
start_prefetch = 30420
log_file = '/Users/felicitia/Desktop/XXX/TopIP/708777107_sorted.csv'
srcIP_idx = 0
desIP_idx = 1
host_idx = 2
url_idx = 3
time_idx = 4

DG = nx.DiGraph()
url_buffer = []
cache = set()
hit_set = set()
prefetch_num = 0
hit_num = 0
training_set = set()
testing_set = set()

def updateDG(url_buffer):
    global DG
    head_node = url_buffer[0]
    successors = set()
    i = 1
    while i < len(url_buffer):
        if url_buffer[i] != head_node:
            successors.add(url_buffer[i])
        i += 1
    for suc in successors:
        if DG.has_edge(head_node, suc):
            old_weight = DG[head_node][suc]['weight']
            head_count = DG.nodes[head_node]['count']
            new_weight = (old_weight*head_count + 1) / (head_count + 1)
            DG[head_node][suc]['weight'] = new_weight
        else: # edge doesn't exist
            if DG.has_node(head_node):
                DG.add_edge(head_node, suc)
                # head_node has outgoing edges
                if 'count' in DG.nodes[head_node]: 
                    DG[head_node][suc]['weight'] = 1 / (DG.nodes[head_node]['count'] + 1)
                else:
                    DG[head_node][suc]['weight'] = 1 # initial weight for new edge (old_weight*head_count + 1) / (head_count + 1)
                    DG.nodes[head_node]['count'] = 0 # initial count for new node
            else:
                DG.add_edge(head_node, suc)
                DG[head_node][suc]['weight'] = 1 # initial weight for new edge (old_weight*head_count + 1) / (head_count + 1)
                DG.nodes[head_node]['count'] = 0 # initial count for new node
                
    # when the whole url_buffer has the same url, then head_node might not have 'count' attribute
    if not DG.has_node(head_node):
        DG.add_node(head_node)
    if len(successors) == 0 and 'count' not in DG.nodes[head_node]:
        DG.nodes[head_node]['count'] = 0
        
    DG.nodes[head_node]['count'] += 1
    return

def drawGraph(dg):
    pos = nx.spring_layout(dg)
    nx.draw_networkx(dg, pos, with_lables=True)
    # node_labels = nx.get_node_attributes(dg,'count')
    # nx.draw_networkx_labels(dg, pos, labels = node_labels)
    edge_labels = nx.get_edge_attributes(dg,'weight')
    nx.draw_networkx_edge_labels(dg, pos, labels = edge_labels)
    plt.savefig("dg.png", format="PNG")
    plt.show()

def prefetch(dg, url, cache):
    global weight_threshold
    global prefetch_num
    if dg.has_node(url):
        for suc in dg.successors(url):
            if dg[url][suc]['weight'] > weight_threshold and (suc not in cache):
                prefetch_num += 1
                cache.add(suc)
                    
start_time = sys_time.time()
# building predictive model dynamically, start prefetching after url_buffer is full    
with open(log_file) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    
    for row in csv_reader:
        actual_url = row[host_idx] + row[url_idx]
        if line_count <= start_prefetch:
            training_set.add(actual_url)
        else:
            testing_set.add(actual_url)
        # add elements to fill the buffer
        if line_count < window_size:
            url_buffer.append(actual_url)
            if line_count == window_size - 1:
                #print('url buffer: ', url_buffer, line_count)
                updateDG(url_buffer)
        
        # update the buffer and prefetching
        else:
            current_url = actual_url
            url_buffer.pop(0)
            url_buffer.append(current_url)
            #print('url buffer: ', url_buffer, line_count)
            updateDG(url_buffer)
            if line_count > start_prefetch:
                if current_url in cache:
                    hit_set.add(current_url)
                    hit_num += 1
                prefetch(DG, current_url, cache)
        line_count += 1
    
    print("--- %s seconds ---" % (sys_time.time() - start_time))
    print('Assumption: no cache staleness')
    print(f'Processed {line_count} lines in {log_file}')
    print('#url in both training set and testing set / #url in testing set = ', 
          len(training_set.intersection(testing_set)) / len(testing_set))
    print('window size:', window_size, 'weight threshold:', weight_threshold, 'start prefetch:', start_prefetch / line_count)
    print('prefetch num = ', prefetch_num)
    print('hit num = ', hit_num)
    print('cache size = ', len(cache))
    print('Precision = ', len(hit_set) / len(cache))
    print('Recall = ', hit_num / (line_count-start_prefetch))
    
#    drawGraph(DG)


--- 1.2334041595458984 seconds ---
Assumption: no cache staleness
Processed 38025 lines in /Users/felicitia/Desktop/CERT_Data/TopIP/708777107_sorted.csv
#url in both training set and testing set / #url in testing set =  0.7274229074889867
window size: 3 weight threshold: 0.8 start prefetch: 0.8
prefetch num =  2880
hit num =  3342
cache size =  2880
Precision =  0.4652777777777778
Recall =  0.43944773175542406


In [23]:
# Dependency Graph (DG) based prefetching technique
# Paper: Using Predictive Prefetching to Improve World Wide Web Latency, 1996
# Assumption: cache is only used once
# Need to computer URL = host + uri (might not be the actual URL b/c some uris already have host and some don't, 
# but it doesn't affect the analysis results b/c the format is the same, e.g., the uri that starts with host name
# always starts with host name)

import networkx as nx
import matplotlib.pyplot as plt
import csv
import time as sys_time


window_size = 3
weight_threshold = 0.8
start_prefetch = 30420
log_file = '/Users/felicitia/Desktop/XXX/TopIP/708777107_sorted.csv'
srcIP_idx = 0
desIP_idx = 1
host_idx = 2
url_idx = 3
time_idx = 4

DG = nx.DiGraph()
url_buffer = []
cache = set()
prefetch_num = 0
hit_num = 0
training_set = set()
testing_set = set()

def updateDG(url_buffer):
    global DG
    head_node = url_buffer[0]
    successors = set()
    i = 1
    while i < len(url_buffer):
        if url_buffer[i] != head_node:
            successors.add(url_buffer[i])
        i += 1
    for suc in successors:
        if DG.has_edge(head_node, suc):
            old_weight = DG[head_node][suc]['weight']
            head_count = DG.nodes[head_node]['count']
            new_weight = (old_weight*head_count + 1) / (head_count + 1)
            DG[head_node][suc]['weight'] = new_weight
        else: # edge doesn't exist
            if DG.has_node(head_node):
                DG.add_edge(head_node, suc)
                # head_node has outgoing edges
                if 'count' in DG.nodes[head_node]: 
                    DG[head_node][suc]['weight'] = 1 / (DG.nodes[head_node]['count'] + 1)
                else:
                    DG[head_node][suc]['weight'] = 1 # initial weight for new edge (old_weight*head_count + 1) / (head_count + 1)
                    DG.nodes[head_node]['count'] = 0 # initial count for new node
            else:
                DG.add_edge(head_node, suc)
                DG[head_node][suc]['weight'] = 1 # initial weight for new edge (old_weight*head_count + 1) / (head_count + 1)
                DG.nodes[head_node]['count'] = 0 # initial count for new node
                
    # when the whole url_buffer has the same url, then head_node might not have 'count' attribute
    if not DG.has_node(head_node):
        DG.add_node(head_node)
    if len(successors) == 0 and 'count' not in DG.nodes[head_node]:
        DG.nodes[head_node]['count'] = 0
        
    DG.nodes[head_node]['count'] += 1
    return

def drawGraph(dg):
    pos = nx.spring_layout(dg)
    nx.draw_networkx(dg, pos, with_lables=True)
    # node_labels = nx.get_node_attributes(dg,'count')
    # nx.draw_networkx_labels(dg, pos, labels = node_labels)
    edge_labels = nx.get_edge_attributes(dg,'weight')
    nx.draw_networkx_edge_labels(dg, pos, labels = edge_labels)
    plt.savefig("dg.png", format="PNG")
    plt.show()

def prefetch(dg, url, cache):
    global weight_threshold
    global prefetch_num
    if dg.has_node(url):
        for suc in dg.successors(url):
            if dg[url][suc]['weight'] > weight_threshold and (suc not in cache):
                prefetch_num += 1
                cache.add(suc)
                    
start_time = sys_time.time()
# building predictive model dynamically, start prefetching after url_buffer is full    
with open(log_file) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    
    for row in csv_reader:
        actual_url = row[host_idx] + row[url_idx]
        if line_count <= start_prefetch:
            training_set.add(actual_url)
        else:
            testing_set.add(actual_url)
        # add elements to fill the buffer
        if line_count < window_size:
            url_buffer.append(actual_url)
            if line_count == window_size - 1:
                #print('url buffer: ', url_buffer, line_count)
                updateDG(url_buffer)
        
        # update the buffer and prefetching
        else:
            current_url = actual_url
            url_buffer.pop(0)
            url_buffer.append(current_url)
            #print('url buffer: ', url_buffer, line_count)
            updateDG(url_buffer)
            if line_count > start_prefetch:
                if current_url in cache:
                    hit_num += 1
                    cache.remove(current_url) # delete cache after used
                prefetch(DG, current_url, cache)
        line_count += 1
        
    print("--- %s seconds ---" % (sys_time.time() - start_time))
    print('Assumption: cache is used once')    
    print(f'Processed {line_count} lines in {log_file}')
    print('#url in both training set and testing set / #url in testing set = ', 
          len(training_set.intersection(testing_set)) / len(testing_set))
    print('window size:', window_size, 'weight threshold:', weight_threshold, 'start prefetch:', start_prefetch / line_count)
    print('prefetch num = ', prefetch_num)
    print('hit num = ', hit_num)
    print('cache size = ', len(cache))
    print('Precision = ', hit_num / prefetch_num)
    print('Recall = ', hit_num / (line_count-start_prefetch))
    

--- 1.066741943359375 seconds ---
Assumption: cache is used once
Processed 38025 lines in /Users/felicitia/Desktop/CERT_Data/TopIP/708777107_sorted.csv
#url in both training set and testing set / #url in testing set =  0.7274229074889867
window size: 3 weight threshold: 0.8 start prefetch: 0.8
prefetch num =  4603
hit num =  2321
cache size =  2282
Precision =  0.5042363675863567
Recall =  0.3051939513477975
